In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [0]:
# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 10
learning_rate = 0.01

In [0]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [9]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
model

RNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/600], Loss: 0.5864
Epoch [1/10], Step [200/600], Loss: 0.1907
Epoch [1/10], Step [300/600], Loss: 0.2306
Epoch [1/10], Step [400/600], Loss: 0.1688
Epoch [1/10], Step [500/600], Loss: 0.2004
Epoch [1/10], Step [600/600], Loss: 0.0374
Epoch [2/10], Step [100/600], Loss: 0.1061
Epoch [2/10], Step [200/600], Loss: 0.0166
Epoch [2/10], Step [300/600], Loss: 0.0094
Epoch [2/10], Step [400/600], Loss: 0.0870
Epoch [2/10], Step [500/600], Loss: 0.0636
Epoch [2/10], Step [600/600], Loss: 0.1455
Epoch [3/10], Step [100/600], Loss: 0.0503
Epoch [3/10], Step [200/600], Loss: 0.1136
Epoch [3/10], Step [300/600], Loss: 0.1358
Epoch [3/10], Step [400/600], Loss: 0.0650
Epoch [3/10], Step [500/600], Loss: 0.1130
Epoch [3/10], Step [600/600], Loss: 0.0829
Epoch [4/10], Step [100/600], Loss: 0.0619
Epoch [4/10], Step [200/600], Loss: 0.0526
Epoch [4/10], Step [300/600], Loss: 0.0284
Epoch [4/10], Step [400/600], Loss: 0.0359
Epoch [4/10], Step [500/600], Loss: 0.0850
Epoch [4/10

In [12]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 


Test Accuracy of the model on the 10000 test images: 97.44 %


In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')